<a href="https://colab.research.google.com/github/Nishasathish13/TheSchoolofAI-END3.0/blob/main/Session%207%20-%20Learning%20Rates%20and%20Evaluation%20Metrics/Assignment/Assignment_Q1/Session_7_Learning_Rates_and_Evaluation_Metrics_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment

Pick any of your past code and:

Implement the following metrics (either on separate models or same, your choice):
* Recall
-Precision
-F1 Score
-BLEU 

1. Perplexity (explain whether you are using bigram, trigram, or something else, what does your PPL score represent?)
2. BERTScore

Once done, proceed to answer questions in the Assignment-Submission Page. 
Questions asked are:

Share the link to the readme file where you have explained all 4 metrics. 
Share the link(s) where we can find the code and training logs for all of your 4 metrics
Share the last 2-3 epochs/stage logs for all of your 4 metrics separately (A, B, C, D) and describe your understanding of the numbers you're seeing, are they good/bad? Why?

# The architecture we are building

![image](https://miro.medium.com/max/1838/1*tXchCn0hBSUau3WO0ViD7w.jpeg)

As we can see here, we will have an encoder, an attention mechanism block and decoder. In the final code the attention mechanicm block and decoder will be merged into single block as we need both to work together. 

As we can see here, we need to create a copy of h1, h2, h3 and h4. These are encoder outputs for a sentence with 4 words. 

# Encoder

We will build our encoder with a GRU, but that's all we know. Let's NOT strait away build a class, but see how to come up with one for the Encoder. We need to answer few questions first:
1. what would be the hidden size of our GRU
2. What would be the input size
3. What would be the embedding dimesions. 

For simplicity, lets keep 1. and 3. to be 256. 

We can't feed our input directly to GRU, we need to tensorize it, convert to embeddings first. 

`embedding = nn.Embedding(input_size, hidden_size) `

## What is input_size?

Remember the line below?

`input_lang, output_lang, pairs = prepareData('eng', 'fra', True)`

#Actual code used

In [1]:
%matplotlib inline
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/question_answer_pairs.txt', encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    line = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Split every line into pairs and normalize
    pairs = [l[1:-3] for l in line]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 20

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH
        # and \p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'eng', True)
print(random.choice(pairs))

Reading lines...
Read 826 sentence pairs
Trimmed to 786 sentence pairs
Counting words...
Counted words:
eng 646
eng 1324
['mid to late summer', 'when do bumblebee colonies reach peak population ?']


In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output, hidden = self.gru(output, hidden)
        output = F.relu(output)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [10]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [11]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))



In [12]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [13]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [14]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [15]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [16]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 45000, print_every=5000)

1m 18s (- 10m 30s) (5000 11%) 3.9134
2m 40s (- 9m 20s) (10000 22%) 3.1923
3m 58s (- 7m 57s) (15000 33%) 2.7551
5m 16s (- 6m 35s) (20000 44%) 2.3226
6m 35s (- 5m 16s) (25000 55%) 2.0564
7m 55s (- 3m 57s) (30000 66%) 1.9253
9m 14s (- 2m 38s) (35000 77%) 1.8719
10m 33s (- 1m 19s) (40000 88%) 1.7635
11m 51s (- 0m 0s) (45000 100%) 1.7989


In [17]:
def Evaluation_data(encoder, decoder):
  target, predicted = [], []
  for i in range(500):
    pair = random.choice(pairs)
    output_words, attentions = evaluate(encoder, decoder, pair[0])
    output_sentence = ' '.join(output_words[:-1])
    
    target.append(pair[1])
    predicted.append(output_sentence)    
  return target, predicted
target, predicted = Evaluation_data(encoder1, attn_decoder1)

# Precision

In [18]:
from sklearn.metrics import precision_score

_DESCRIPTION = """
Precision is the fraction of the true examples among the predicted examples. 
It can be computed with: Precision = TP / (TP + FP)
TP: True positive
FN: False negative
"""

def final_Evaluation_data(target, predicted):
  y_true, y_pred = [], []

  for t, p in zip(target, predicted):
    y_true.append(0)
    if(t==p):
      y_pred.append(0)
    else:
      y_pred.append(1)
  return y_true, y_pred

y_true, y_pred = final_Evaluation_data(target, predicted)

#recall_metric = datasets.load_metric("recall")
results = precision_score(y_pred, y_true, average=None, zero_division=0)
print(results)

[0.332 0.   ]


# Recall

In [19]:
from sklearn.metrics import recall_score

_DESCRIPTION = """
Recall is the fraction of the total amount of relevant examples that were actually retrieved. It can be computed with:
Recall = TP / (TP + FN)
TP: True positive
FN: False negative
"""

def final_Evaluation_data(target, predicted):
  y_true, y_pred = [], []

  for t, p in zip(target, predicted):
    y_true.append(0)
    if(t==p):
      y_pred.append(0)
    else:
      y_pred.append(1)
  return y_true, y_pred

y_true, y_pred = final_Evaluation_data(target, predicted)

#recall_metric = datasets.load_metric("recall")
results = recall_score(y_pred, y_true, average=None, zero_division=0)
print(results)

[1. 0.]


# F1 score

In [20]:
from sklearn.metrics import f1_score

_DESCRIPTION = """
The F1 score is the harmonic mean of the precision and recall. It can be computed with:
F1 = 2 * (precision * recall) / (precision + recall)
"""
results = f1_score(y_pred, y_true, average=None)
print(results)


[0.4984985 0.       ]


# Perplexity (PPL score)

In [21]:
#https://colab.research.google.com/github/huggingface/notebooks/blob/master/transformers_doc/pytorch/perplexity.ipynb#scrollTo=i5aSbBg1Jg1-
#https://medium.com/unpackai/perplexity-and-accuracy-in-classification-114b57bd820d#:~:text=We%20know%20that%20the%20accuracy,exponential%20of%20the%20loss%20function.

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f , perplexity %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg, math.exp(print_loss_avg)))
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


In [28]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 45000, print_every=5000)

1m 55s (- 15m 27s) (5000 11%) 3.9375 , perplexity 51.2883
3m 47s (- 13m 16s) (10000 22%) 3.2334 , perplexity 25.3651
5m 40s (- 11m 21s) (15000 33%) 2.7264 , perplexity 15.2774
7m 35s (- 9m 29s) (20000 44%) 2.3630 , perplexity 10.6233
9m 32s (- 7m 37s) (25000 55%) 2.0888 , perplexity 8.0749
11m 28s (- 5m 44s) (30000 66%) 1.9400 , perplexity 6.9586
13m 24s (- 3m 49s) (35000 77%) 1.9229 , perplexity 6.8409
15m 20s (- 1m 55s) (40000 88%) 1.7875 , perplexity 5.9743
17m 18s (- 0m 0s) (45000 100%) 1.8359 , perplexity 6.2709


# BLEU score

The Bilingual Evaluation Understudy Score, or BLEU for short, is a metric for evaluating a generated sentence to a reference sentence.

A perfect match results in a score of 1.0, whereas a perfect mismatch results in a score of 0.0.

In [27]:
#https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
from torchtext.data.metrics import bleu_score

def Evaluation_data(encoder, decoder):
  target, predicted = [], []
  for i in range(500):
    pair = random.choice(pairs)
    output_words, attentions = evaluate(encoder, decoder, pair[0])
    output_sentence = ' '.join(output_words[:-1])
    
    target.append(pair[1])
    predicted.append(output_sentence)
    # print(target)
    
  return target, predicted
  

target, predicted = Evaluation_data(encoder1, attn_decoder1)

# for targ in target:
#   print(targ.split())
  #print(type(targ))

candidate_corpus = []
candidate_corpus.append(targ.split())
#print(candidate_corpus)

# for pred in predicted:
#   print(pred.split())
  #print(type(pred))

references_corpus = []
references_corpus.append(pred.split())

print(candidate_corpus)
print(references_corpus)

bleu_score(candidate_corpus, references_corpus)

[['what', 'is', 'an', 'otter', 's', 'den', 'called', '?']]
[['what', 'is', 'an', 'otter', 's', 'den', 'called', '?']]


0.0

In [24]:
#https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

from torchtext.data.metrics import bleu_score

def prepareDataForEvaluation(target, predicted):
  candidate_corpus, references_corpus = [], []

  for t, p in zip(target, predicted):
    candidate_corpus.append(t.split(" "))
    references_corpus.append([p.split(" ")])
  
  return candidate_corpus, references_corpus

candidate_corpus, references_corpus = prepareDataForEvaluation(target, predicted)
bleu_score(candidate_corpus, references_corpus)


0.3764677047729492

# BERT score

In [28]:
!pip install bert-score

from bert_score import BERTScorer
bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)

     |████████████████████████████████| 60 kB 8.0 MB/s 
     |████████████████████████████████| 3.4 MB 42.3 MB/s 
     |████████████████████████████████| 895 kB 61.7 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 74.4 MB/s 
     |████████████████████████████████| 3.3 MB 61.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
def prepareDataForEvaluation(target, predicted):
  hyps, refs = [], []

  for t, p in zip(target, predicted):
    hyps.append(t)
    refs.append([p])
  
  return hyps, refs

hyps, refs = prepareDataForEvaluation(target, predicted)
P, R, F1 = bert_scorer.score(hyps, refs)
print(P.mean(), R.mean(), F1.mean())

tensor(0.4370) tensor(0.4471) tensor(0.4422)
